# Frnech

In [1]:
! pip install numpy
! pip install spacy
! pip install nltk
! pip install datasets
! pip install stanza

In [2]:
!python -m spacy download fr_core_news_sm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 1.7 MB/s  0:00:09m0:00:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


### Load Dataset

In [3]:
from datasets import load_dataset
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize  # tokenization

nltk.download('punkt')       # Tokenizer model
nltk.download('punkt_tab')

nltk.download('stopwords')   # List of common stopwords for text normalization
from nltk.corpus import stopwords

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/park/miniforge3/envs/nlp_clean/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/park/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/park/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/park/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
dataset_fr = load_dataset("wikimedia/wikipedia", "20231101.fr")

In [5]:
df_fr = pd.DataFrame(dataset_fr["train"])
df_sample_fr = df_fr.sample(n=1000, random_state=42)
df_sample_fr.head()

,id,url,title,text
836606,4156881,https://fr.wikipedia.org/wiki/Ko%C5%A1ick%C3%A...,Košické Oľšany,Košické Oľšany est un village de Slovaquie sit...
2481962,9978312,https://fr.wikipedia.org/wiki/Star%20Trek%3A%2...,Star Trek: The Rebel Universe,Star Trek: The Rebel Universe est un jeu vidéo...
125800,328808,https://fr.wikipedia.org/wiki/Fossemanant,Fossemanant,Fossemanant est une commune française située d...
40695,99200,https://fr.wikipedia.org/wiki/Canton%20de%20Ne...,Canton de Neung-sur-Beuvron,Le canton de Neung-sur-Beuvron est un ancien c...
1200964,6135961,https://fr.wikipedia.org/wiki/A%20Real%20Live%...,A Real Live Dead One,est une compilation du groupe de heavy metal b...


### Removing punctuations and symbols

In [6]:
def clean_text_fr(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)  # Remove newline characters
    text = re.sub(r'[^A-Za-zÀÂÆÇÉÈÊËÎÏÔŒÙÛÜàâæçéèêëîïôœùûüÿ0-9\s\'\-]', '', text) # Keep French characters and frequent used symbol
    text = re.sub(r'\s+', ' ', text).strip()  # multiple spaces
    return text

df_sample_fr["clean_text"] = df_sample_fr["text"].apply(clean_text_fr)

### Tokenization

In [7]:
import spacy
nlp = spacy.load("fr_core_news_sm")

df_sample_fr["tokens"] = df_sample_fr["clean_text"].apply(
    lambda x: [token.text for token in nlp(x)]
)

In [8]:
df_sample_fr.head()

,id,url,title,text,clean_text,tokens
836606,4156881,https://fr.wikipedia.org/wiki/Ko%C5%A1ick%C3%A...,Košické Oľšany,Košické Oľšany est un village de Slovaquie sit...,koické oany est un village de slovaquie situé ...,"[koické, oany, est, un, village, de, slovaquie..."
2481962,9978312,https://fr.wikipedia.org/wiki/Star%20Trek%3A%2...,Star Trek: The Rebel Universe,Star Trek: The Rebel Universe est un jeu vidéo...,star trek the rebel universe est un jeu vidéo ...,"[star, trek, the, rebel, universe, est, un, je..."
125800,328808,https://fr.wikipedia.org/wiki/Fossemanant,Fossemanant,Fossemanant est une commune française située d...,fossemanant est une commune française située d...,"[fossemanant, est, une, commune, française, si..."
40695,99200,https://fr.wikipedia.org/wiki/Canton%20de%20Ne...,Canton de Neung-sur-Beuvron,Le canton de Neung-sur-Beuvron est un ancien c...,le canton de neung-sur-beuvron est un ancien c...,"[le, canton, de, neung-sur-beuvron, est, un, a..."
1200964,6135961,https://fr.wikipedia.org/wiki/A%20Real%20Live%...,A Real Live Dead One,est une compilation du groupe de heavy metal b...,est une compilation du groupe de heavy metal b...,"[est, une, compilation, du, groupe, de, heavy,..."


### Lemmatization

In [9]:
def lemmatize_tokens_spacy(tokens):
    doc = nlp(' '.join(tokens))
    return [token.lemma_ for token in doc]

df_sample_fr['tokens_lemmatized'] = df_sample_fr['tokens'].apply(lemmatize_tokens_spacy)

In [10]:
df_sample_fr.head()

,id,url,title,text,clean_text,tokens,tokens_lemmatized
836606,4156881,https://fr.wikipedia.org/wiki/Ko%C5%A1ick%C3%A...,Košické Oľšany,Košické Oľšany est un village de Slovaquie sit...,koické oany est un village de slovaquie situé ...,"[koické, oany, est, un, village, de, slovaquie...","[koické, oany, être, un, village, de, slovaqui..."
2481962,9978312,https://fr.wikipedia.org/wiki/Star%20Trek%3A%2...,Star Trek: The Rebel Universe,Star Trek: The Rebel Universe est un jeu vidéo...,star trek the rebel universe est un jeu vidéo ...,"[star, trek, the, rebel, universe, est, un, je...","[star, trek, the, rebel, universe, être, un, j..."
125800,328808,https://fr.wikipedia.org/wiki/Fossemanant,Fossemanant,Fossemanant est une commune française située d...,fossemanant est une commune française située d...,"[fossemanant, est, une, commune, française, si...","[fossemaner, être, un, commune, français, situ..."
40695,99200,https://fr.wikipedia.org/wiki/Canton%20de%20Ne...,Canton de Neung-sur-Beuvron,Le canton de Neung-sur-Beuvron est un ancien c...,le canton de neung-sur-beuvron est un ancien c...,"[le, canton, de, neung-sur-beuvron, est, un, a...","[le, canton, de, neung-sur-beuvron, être, un, ..."
1200964,6135961,https://fr.wikipedia.org/wiki/A%20Real%20Live%...,A Real Live Dead One,est une compilation du groupe de heavy metal b...,est une compilation du groupe de heavy metal b...,"[est, une, compilation, du, groupe, de, heavy,...","[être, un, compilation, de, groupe, de, heavy,..."


### Normalization

In [11]:
def load_stopwords(language: str):
    filename = f"stopwords/stopwords-{language}.txt"
    try:
        with open(filename, "r", encoding="utf-8") as f:
            stopwords = set(line.strip() for line in f if line.strip())
        return stopwords
    except FileNotFoundError:
        print(f"'{language}' - language not supported")
        return set()
    
stopwords_french = load_stopwords("fr")

In [12]:
# Token Normalization Function (Stopword Removal)
stopwords_dict = {
    # "ko": set(stopwords_korean),
    # "pt": set(stopwords_portuguese),
    # "it": set(stopwords_italian),
    # "en": set(stopwords_english),
    "fr": set(stopwords_french),
}

def normalize_tokens(tokens, lang):
    if lang in stopwords_dict:
        stops = stopwords_dict[lang]
    else:
        from nltk.corpus import stopwords
        try:
            stops = set(stopwords.words(lang))  # NLTK stopwords for other languages
        except:
            stops = set()  # Fallback if stopwords are unavailable
    # Remove stopwords and single-character tokens
    normalized_tokens = [w for w in tokens if w not in stops and len(w) > 1]

    return normalized_tokens

# Apply Stopword Removal to the Pre-Tokenized Dataset
df_sample_fr["tokens_norm"] = df_sample_fr["tokens_lemmatized"].apply(lambda x: normalize_tokens(x, lang='fr'))

df_sample_fr.head()

,id,url,title,text,clean_text,tokens,tokens_lemmatized,tokens_norm
836606,4156881,https://fr.wikipedia.org/wiki/Ko%C5%A1ick%C3%A...,Košické Oľšany,Košické Oľšany est un village de Slovaquie sit...,koické oany est un village de slovaquie situé ...,"[koické, oany, est, un, village, de, slovaquie...","[koické, oany, être, un, village, de, slovaqui...","[koické, oany, village, slovaquie, situer, rég..."
2481962,9978312,https://fr.wikipedia.org/wiki/Star%20Trek%3A%2...,Star Trek: The Rebel Universe,Star Trek: The Rebel Universe est un jeu vidéo...,star trek the rebel universe est un jeu vidéo ...,"[star, trek, the, rebel, universe, est, un, je...","[star, trek, the, rebel, universe, être, un, j...","[star, trek, the, rebel, universe, jeu, vidéo,..."
125800,328808,https://fr.wikipedia.org/wiki/Fossemanant,Fossemanant,Fossemanant est une commune française située d...,fossemanant est une commune française située d...,"[fossemanant, est, une, commune, française, si...","[fossemaner, être, un, commune, français, situ...","[fossemaner, commune, français, situer, départ..."
40695,99200,https://fr.wikipedia.org/wiki/Canton%20de%20Ne...,Canton de Neung-sur-Beuvron,Le canton de Neung-sur-Beuvron est un ancien c...,le canton de neung-sur-beuvron est un ancien c...,"[le, canton, de, neung-sur-beuvron, est, un, a...","[le, canton, de, neung-sur-beuvron, être, un, ...","[canton, neung-sur-beuvron, ancien, canton, fr..."
1200964,6135961,https://fr.wikipedia.org/wiki/A%20Real%20Live%...,A Real Live Dead One,est une compilation du groupe de heavy metal b...,est une compilation du groupe de heavy metal b...,"[est, une, compilation, du, groupe, de, heavy,...","[être, un, compilation, de, groupe, de, heavy,...","[compilation, groupe, heavy, metal, britanniqu..."


### Save File

In [13]:
import os

def save_lang_dataset(df, lang: str, save_dir: str = "data"):
    required_cols = ["text", "clean_text", "tokens", "tokens_lemmatized", "tokens_norm"]
    
    # add language label
    df_to_save = df[required_cols].copy()
    df_to_save["label"] = lang
    
    os.makedirs(save_dir, exist_ok=True)
    output_path = os.path.join(save_dir, f"output_{lang}.parquet")  
    df_to_save.to_parquet(output_path, index=False)
    display(df_to_save)


In [14]:
save_lang_dataset(df_sample_fr, lang="fr")

,text,clean_text,tokens,tokens_lemmatized,tokens_norm,label
836606,Košické Oľšany est un village de Slovaquie sit...,koické oany est un village de slovaquie situé ...,"[koické, oany, est, un, village, de, slovaquie...","[koické, oany, être, un, village, de, slovaqui...","[koické, oany, village, slovaquie, situer, rég...",fr
2481962,Star Trek: The Rebel Universe est un jeu vidéo...,star trek the rebel universe est un jeu vidéo ...,"[star, trek, the, rebel, universe, est, un, je...","[star, trek, the, rebel, universe, être, un, j...","[star, trek, the, rebel, universe, jeu, vidéo,...",fr
125800,Fossemanant est une commune française située d...,fossemanant est une commune française située d...,"[fossemanant, est, une, commune, française, si...","[fossemaner, être, un, commune, français, situ...","[fossemaner, commune, français, situer, départ...",fr
40695,Le canton de Neung-sur-Beuvron est un ancien c...,le canton de neung-sur-beuvron est un ancien c...,"[le, canton, de, neung-sur-beuvron, est, un, a...","[le, canton, de, neung-sur-beuvron, être, un, ...","[canton, neung-sur-beuvron, ancien, canton, fr...",fr
1200964,est une compilation du groupe de heavy metal b...,est une compilation du groupe de heavy metal b...,"[est, une, compilation, du, groupe, de, heavy,...","[être, un, compilation, de, groupe, de, heavy,...","[compilation, groupe, heavy, metal, britanniqu...",fr
...,...,...,...,...,...,...
2233978,"En 2023, Le Festival international du film pol...",en 2023 le festival international du film poli...,"[en, 2023, le, festival, international, du, fi...","[en, 2023, le, festival, international, de, fi...","[2023, festival, international, film, politiqu...",fr
579271,"Gaspard Auguste Brullé, un entomologiste franç...",gaspard auguste brullé un entomologiste frança...,"[gaspard, auguste, brullé, un, entomologiste, ...","[gaspard, auguste, bruller, un, entomologiste,...","[gaspard, auguste, bruller, entomologiste, fra...",fr
1790556,est un astéroïde de la ceinture principale d'a...,est un astéroïde de la ceinture principale d'a...,"[est, un, astéroïde, de, la, ceinture, princip...","[être, un, astéroïde, de, le, ceinture, princi...","[astéroïde, ceinture, principal, astéroïde, dé...",fr
602032,Le Mulhouse Handball Sud Alsace était un club ...,le mulhouse handball sud alsace était un club ...,"[le, mulhouse, handball, sud, alsace, était, u...","[le, mulhouse, handball, sud, alsace, être, un...","[mulhouse, handball, sud, alsace, club, handba...",fr
